# Prices

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### CPI components 


https://www.bls.gov/cpi/tables/relative-importance/home.htm

In [66]:
base = 'CUUR0000'

series1 = {'SA0': 'All items',
           'SAM': 'Medical care',
           'SAH': 'Housing',
           'SEFV': 'Food away from home',
           'SAF11': 'Food at home',
           'SAR': 'Recreation',
           'SAE1': 'Education',
           'SAT': 'Transportation',
           'SAA': 'Apparel',
           'SA0E': 'Energy',
           'SAE2': 'Communication'}

series2 = {'SEHA': 'Rent of primary residence',
           'SEHC': "Owners' equivalent rent of primary residence",
           'SAH3': 'Household furnishings and operations',
           'SETA01': 'New vehicles',
           'SETA02': 'Used cars and trucks',
           'SEMD': 'Hospital and related services',
           'SEMC': 'Professional services',
           'SEME': 'Health insurance'}

series = {**series1, **series2}
series = {base+key: value for key, value in series.items()}

url = 'https://www.bls.gov/cpi/tables/relative-importance/2019.txt'
r = requests.get(url)
data = r.text

wgts = {value: re.findall(value+r'\..*\s+(\d+\.\d+)\s+\d+\.\d+\s+\r', data)[0] 
        for key, value in series.items()}

wgt_date = pd.to_datetime('2019-12-01')

In [67]:
# Start year and end year
years = (2017, 2020)
df = bls_api(series, years, bls_key)
df.to_csv(data_dir / 'cpi_comp_raw.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [72]:
df = pd.read_csv(data_dir / 'cpi_comp_raw.csv', index_col='date')
df.index = pd.to_datetime(df.index)
dates = {'Latest': df.index[-1], 'Previous': df.index[-13]}

d = {}

for name, date in dates.items():

    updated_wgts = {srs: ((df.loc[date, srs] / df.loc[wgt_date, srs])
                     * float(wgts[srs]) / 
                     (df.loc[date, 'All items'] / 
                      df.loc[wgt_date, 'All items']))
                    for key, srs in series.items()}
    
    d[name] = {srs: (df[srs].pct_change(12).loc[date] 
                        * (updated_wgts[srs]))
                  for key, srs in series.items()}
    
results = pd.DataFrame(d).round(2).sort_values('Latest', ascending=False)
results.loc[series1.values()].drop('All items').to_csv(data_dir / 'cpi_comp.csv', index_label='name')

write_txt(text_dir / 'cpi_mo1.txt', dtxt(df.index[-1])['mon2'])
write_txt(text_dir / 'cpi_mo2.txt', dtxt(df.index[-13])['mon2'])

results

,Latest,Previous
All items,1.31,1.75
Housing,0.89,1.19
Owners' equivalent rent of primary residence,0.62,0.76
Medical care,0.40,0.30
Food at home,0.36,0.04
Rent of primary residence,0.23,0.29
Health insurance,0.22,0.20
Food away from home,0.22,0.20
Household furnishings and operations,0.12,0.10
Used cars and trucks,0.11,0.06


In [74]:
ltdate = dtxt(df.index[-1])['mon1']
prdate = dtxt(df.index[-13])['mon1']

data = results.loc[series1.values()].drop('All items')
c1n = data.index[0].lower()
c1v = data.Latest.iloc[0]
c1vt = value_text(c1v, style="contribution_to", ptype="pp", digits=2, casual=True)
c1 = f'In {ltdate}, {c1n} {c1vt} overall CPI inflation'
p1v = data.Previous.iloc[0]
ct1 = compare_text(c1v, p1v, cutoffs=[0.05, 0.3, 1])
p1vt = (value_text(p1v, style='contribution_of', ptype='pp', digits=2)
        .replace('a ', f"the category's {prdate} "))

c2n = data.index[1]
c2v = data.Latest.iloc[1]
c2vt = value_text(c2v, style="contribution_to", ptype="pp", digits=2)
c2 = f'{c2n} {c2vt} inflation'
p2v = data.Previous.iloc[1]
if abs(p2v - c2v) > 0.2:
    ct2 = compare_text(c2v, p2v, cutoffs=[0.05, 0.3, 1])
else:
    ct2 = 'and'
p2vt = value_text(p2v, style='contribution', ptype='pp', digits=2, casual=True)

c5n = data.index[2]
c5v = data.Latest.iloc[2]
c5vt = value_text(c5v, style="contribution_to", ptype="pp", digits=2)
c5 = f'{c5n} {c5vt} inflation'
p5v = data.Previous.iloc[2]
if abs(p5v - c5v) > 0.2:
    ct5 = compare_text(c5v, p5v, cutoffs=[0.05, 0.3, 1])
    style = 'contribution_of'
else:
    ct5 = 'and'
    style = 'contribution'
p5vt = value_text(p5v, style=style, ptype='pp', digits=2, casual=True)

c3n = data.index[-1]
c3v = data.Latest.iloc[-1]
c3vt = value_text(c3v, style="contribution_to", ptype="pp", digits=2)
c3 = f'{c3n} {c3vt} inflation in {ltdate}'
p3v = data.Previous.iloc[-1]
ct3 = compare_text(c3v, p3v, cutoffs=[0.05, 0.3, 1])
p3vt = (value_text(p3v, style='contribution_of', ptype='pp', digits=2)
        .replace('a ', 'the ').replace('an ', 'the '))

c4n = data.index[-2]
c4v = data.Latest.iloc[-2]
c4vt = value_text(c4v, style="contribution", ptype="pp", digits=2, casual=True)
c4 = f'{c4n} {c4vt} in {ltdate}'
p4v = data.Previous.iloc[-2]
ct4 = compare_text(c4v, p4v, cutoffs=[0.05, 0.3, 1])
p4vt = (value_text(p4v, style='contribution_of', ptype='pp', digits=2, casual=True)
        .replace('a ', 'the ').replace('an ', 'the '))

text = (f'{c1}, {ct1} {p1vt}. {c2} in {ltdate} {ct2} {p2vt} in {prdate}. '+
        f'{c5}, {ct5} {p5vt} one year prior.\\\\\n\n'+
        f'{c3}, {ct3} {p3vt} during the same month one year prior. '+
        f'{c4}, {ct4} {p4vt} in {prdate}.')
write_txt(text_dir / 'cpicomp.txt', text)
print(text)

In August 2020, medical care added 0.40 percentage point to overall CPI inflation, slightly above the category's August 2019 contribution of 0.30 percentage point. Housing contributed 0.89 percentage point to inflation in August 2020 slightly below added 1.19 percentage points in August 2019. Food away from home contributed 0.22 percentage point to inflation, and added 0.20 percentage point one year prior.\\

Communication contributed 0.10 percentage point to inflation in August 2020, slightly above the subtraction of 0.04 percentage point during the same month one year prior. Energy subtracted 0.56 percentage point in August 2020, slightly below the reduction of 0.30 percentage point in August 2019.


### CPI Table

In [141]:
df = pd.read_csv(data_dir / 'cpi_comp_raw.csv', index_col='date')
df.index = pd.to_datetime(df.index)
ltdt = df.index[-1]

data = df.pct_change(12) * 100

tbl = pd.DataFrame()
for i in [-1, -2, -3, -4, -13, -25]:
    tbl[df.index[i]] = data.iloc[i]
    
dt = '2020-02-01'
cdt = pd.to_datetime(dt)
tbl[cdt] = data.loc[dt]

tbl = tbl[tbl.columns.sort_values(ascending=False)]
tbl = tbl.applymap('{:,.1f}'.format)
tbl.columns = [dtxt(col)['mon6'] for col in tbl.columns]

wgt_dt = dtxt(ltdt)['mon6']
wgt_col = f'Weight, {wgt_dt}'
tbl[wgt_col] = pd.Series(updated_wgts).astype('float').map('{:,.3f}'.format)
tbl.loc['All items', wgt_col] = '100.0'

order = ['All items', 'Housing', "Owners' equivalent rent of primary residence",
         'Rent of primary residence', 'Household furnishings and operations',  
         'Transportation', 'New vehicles',
         'Used cars and trucks', 'Medical care', 'Professional services',
         'Hospital and related services', 'Health insurance',
         'Food at home', 'Food away from home', 'Energy', 'Recreation',
         'Communication', 'Education', 'Apparel']
tbl = tbl.loc[order]
s2adj = {val: f'\hspace{{2mm}} {val}' for val in list(series2.values())}
s1 = {val: val for val in series1.values()}
adj = {**s2adj, **s1}
tbl.index = tbl.index.map(adj)
tbl = (tbl.rename({"\hspace{2mm} Owners' equivalent rent of primary residence": 
                   "\hspace{2mm} Owners' equivalent rent"})
          .rename({'\hspace{2mm} Household furnishings and operations':
                   '\hspace{2mm} Household furnishings \& ops.'}))
(tbl.to_csv(data_dir / 'cpi_comp.tex', sep='&', 
           line_terminator='\\\ ', quotechar=' '))
tbl

,Aug `20,Jul `20,Jun `20,May `20,Feb `20,Aug `19,Aug `18,"Weight, Aug `20"
All items,1.3,1.0,0.6,0.1,2.3,1.7,2.7,100.0
Housing,2.1,2.0,2.0,2.1,2.7,2.8,2.9,42.022
\hspace{2mm} Owners' equivalent rent,2.7,2.8,2.8,3.1,3.3,3.3,3.3,22.655
\hspace{2mm} Rent of primary residence,2.9,3.1,3.2,3.5,3.8,3.7,3.6,7.708
\hspace{2mm} Household furnishings \& ops.,2.5,1.5,1.4,1.6,0.7,2.2,0.8,4.621
Transportation,-4.1,-5.9,-8.2,-11.1,1.7,-1.1,6.4,15.960
\hspace{2mm} New vehicles,0.7,0.5,-0.2,-0.3,0.4,0.2,0.3,3.746
\hspace{2mm} Used cars and trucks,4.0,-0.9,-2.8,-0.4,-1.3,2.1,1.3,2.665
Medical care,4.5,5.0,5.1,4.9,4.6,3.5,1.5,8.695
\hspace{2mm} Professional services,2.3,2.5,2.2,2.2,1.2,1.4,0.8,3.635


In [140]:
tbl.loc['All items', wgt_col] = '100.0'

In [142]:
ltdate = dtxt(ltdt)['mon1']

res = data.loc[[ltdt, cdt]].T.sort_values(by=[ltdt])

hc = res.loc['Housing', ltdt]
h1 = value_text(res.loc['Housing', ltdt])
hp = res.loc['Housing', cdt]
hch = compare_text(hc, hp, [0.3, 1.0, 3.0])
m1 = value_text(res.loc['Medical care', ltdt])
mpr = value_text(res.loc['Medical care', cdt], casual=True)
fah1 = value_text(res.loc['Food at home', ltdt])
fahpr = res.loc['Food at home', cdt]
tc = res.loc['Transportation', ltdt]
t1 = value_text(tc)
tp = res.loc['Transportation', cdt]
tpr = value_text(tp, style='increase_end').replace('a ', '').replace('an ', '')
tch = compare_text(tc, tp, [0.3, 1.0, 3.0])
e1 = value_text(res.loc['Energy', ltdt])
epr = value_text(res.loc['Energy', cdt], style='increase_end')

text = (f'Housing prices {h1} over the year ending {ltdate}, '+
        f'{hch} the pre-COVID rate of {hp:.1f} percent (covering the year '+
        f'ending February 2020). Medical care prices {m1}, '+
        f'these prices {mpr} over the year ending February 2020. '+
        'In contrast, prices of food consumed at home '+
        f'(groceries) {fah1} in the year ending '+
        f'{ltdate} compared to {fahpr:.1f} percent over the year '+
        'ending February 2020. \\\\\n\n'+
        f'Transportation prices {t1} over the year ending {ltdate}, '+
        f'{tch} the pre-COVID {tpr}. '+
        f'Energy prices {e1} in the latest month, '+
        f'compared to {epr} in February 2020. Energy prices '+
        'are historically more volatile than other categories. ')
write_txt(text_dir / 'cpicomp2.txt', text)
print(text)

Housing prices increased 2.1 percent over the year ending August 2020, slightly below the pre-COVID rate of 2.7 percent (covering the year ending February 2020). Medical care prices increased 4.5 percent, these prices grew 4.6 percent over the year ending February 2020. In contrast, prices of food consumed at home (groceries) increased 4.6 percent in the year ending August 2020 compared to 0.8 percent over the year ending February 2020. \\

Transportation prices decreased 4.1 percent over the year ending August 2020, far below the pre-COVID 1.7 percent increase. Energy prices decreased nine percent in the latest month, compared to a 2.8 percent increase in February 2020. Energy prices are historically more volatile than other categories. 


In [128]:
compare_text(tc, tp, [0.3, 1.0, 3.0])

'far below'

In [ ]:
value_text()

### CPI Decomposition

In [ ]:
# Relative weights for series of interest, from here: 
# https://www.bls.gov/cpi/tables/relative-importance/home.htm
rel_wgt = {'CUUR0000SAF1': [(('2009-12-01', '2011-12-01'), 13.738),
                           (('2011-12-01', '2013-12-01'), 14.308),
                           (('2013-12-01', '2015-12-01'), 13.891), 
                           (('2015-12-01', '2017-12-01'), 14.015), 
                           (('2017-12-01', '2019-12-01'), 13.384),
                           (('2019-12-01', '2021-12-01'), 13.771)],
           'CUUR0000SA0': [(('2009-12-01', '2011-12-01'), 100.0),
                           (('2011-12-01', '2013-12-01'), 100.0),
                           (('2013-12-01', '2015-12-01'), 100.0), 
                           (('2015-12-01', '2017-12-01'), 100.0), 
                           (('2017-12-01', '2019-12-01'), 100.0),
                           (('2019-12-01', '2021-12-01'), 100.0)],
           'CUUR0000SA0E': [(('2009-12-01', '2011-12-01'), 8.553),
                            (('2011-12-01', '2013-12-01'), 9.679),
                            (('2013-12-01', '2015-12-01'), 9.046), 
                            (('2015-12-01', '2017-12-01'), 6.816), 
                            (('2017-12-01', '2019-12-01'), 7.513),
                            (('2019-12-01', '2021-12-01'), 6.706)],
           'CUUR0000SAH1': [(('2009-12-01', '2011-12-01'), 32.289),
                            (('2011-12-01', '2013-12-01'), 31.539),
                            (('2013-12-01', '2015-12-01'), 32.029), 
                            (('2015-12-01', '2017-12-01'), 33.15), 
                            (('2017-12-01', '2019-12-01'), 32.843),
                            (('2019-12-01', '2021-12-01'), 33.158)],
           'CUUR0000SACL1E': [(('2009-12-01', '2011-12-01'), 21.276),
                              (('2011-12-01', '2013-12-01'), 19.852),
                              (('2013-12-01', '2015-12-01'), 19.71), 
                              (('2015-12-01', '2017-12-01'), 19.613), 
                              (('2017-12-01', '2019-12-01'), 19.849),
                              (('2019-12-01', '2021-12-01'), 20.137)],
           'CUUR0000SASLE': [(('2009-12-01', '2011-12-01'), 56.432),
                             (('2011-12-01', '2013-12-01'), 56.161),
                             (('2013-12-01', '2015-12-01'), 57.353), 
                             (('2015-12-01', '2017-12-01'), 59.556), 
                             (('2017-12-01', '2019-12-01'), 59.254),
                             (('2019-12-01', '2021-12-01'), 59.387)]}


series = {key: key for key, value in rel_wgt.items()}

# Start year and end year
years = (2008, 2020)
df = bls_api(series, years, bls_key)

# Dictionary combining all the info for each series
d = {i: {'name': i,
         'values': df[i],
         'rel_wgt': rel_wgt[i]} for i in list(rel_wgt.keys())}

# Adjust for changes to relative importance
df1, df2, df3, df4, df5, df6 = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
for i, v in d.items():
    start, end = v['rel_wgt'][0][0][0], v['rel_wgt'][0][0][1]
    rwc, rwn = v['rel_wgt'][0][1], v['rel_wgt'][1][1]
    df1.at[start: end, i] = (v['values'].loc[start: end])
    df1[i] = (df1[i].diff().cumsum() / df1.loc[start, i] + 1)
    df1.at[start, i] = 1.0
    df1[i] = (df1[i] * rwc)
    link = (df1.loc[end, i] / rwn)
    
    # Next set of dates
    start, end = v['rel_wgt'][1][0][0], v['rel_wgt'][1][0][1]
    rwc, rwn = v['rel_wgt'][1][1], v['rel_wgt'][2][1]
    df2[i] = (v['values'].loc[start: end])
    df2[i] = df2[i].diff().cumsum() / df2.loc[start, i] + 1
    df2.at[start, i] = 1.0
    df2[i] = (df2[i] * rwc) * link
    link = (df2.loc[end, i] / rwn)
    
    # Next set of dates
    start, end = v['rel_wgt'][2][0][0], v['rel_wgt'][2][0][1]
    rwc, rwn = v['rel_wgt'][2][1], v['rel_wgt'][3][1]
    df3[i] = (v['values'].loc[start: end])
    df3[i] = df3[i].diff().cumsum() / df3.loc[start, i] + 1
    df3.at[start, i] = 1.0
    df3[i] = (df3[i] * rwc) * link
    link = (df3.loc[end, i] / rwn)
    
    # Next set of dates
    start, end = v['rel_wgt'][3][0][0], v['rel_wgt'][3][0][1]
    rwc, rwn = v['rel_wgt'][3][1], v['rel_wgt'][4][1]
    df4[i] = (v['values'].loc[start: end])
    df4[i] = df4[i].diff().cumsum() / df4.loc[start, i] + 1
    df4.at[start, i] = 1.0
    df4[i] = (df4[i] * rwc) * link
    link = (df4.loc[end, i] / rwn)

    # Next set of dates
    start, end = v['rel_wgt'][4][0][0], v['rel_wgt'][4][0][1]
    rwc, rwn = v['rel_wgt'][4][1], v['rel_wgt'][5][1]
    df5[i] = (v['values'].loc[start: end])
    df5[i] = df5[i].diff().cumsum() / df5.loc[start, i] + 1
    df5.at[start, i] = 1.0
    df5[i] = (df5[i] * rwc) * link
    link = (df5.loc[end, i] / rwn)    
    
    # Next set of dates
    start, end = v['rel_wgt'][5][0][0], v['rel_wgt'][5][0][1]
    rwc = v['rel_wgt'][5][1]
    df6[i] = (v['values'].loc[start: end])
    df6[i] = df6[i].diff().cumsum() / df6.loc[start, i] + 1
    df6.at[start, i] = 1.0
    df6[i] = (df6[i] * rwc) * link
    
# Calculate exactly what I want to show
final = pd.concat([df1, df2, df3, df4, df5, df6])  # Combine the various relative importance dfs
final = final[~final.index.duplicated(keep='first')].diff(12) # Drop duplicate pivot year data
# Core services is services less food, energy, and shelter
final['core_services'] = final['CUUR0000SASLE'] - final['CUUR0000SAH1']
# Combine food and energy
final['food_energy'] = final['CUUR0000SAF1'] + final['CUUR0000SA0E']
final = final.dropna().round(2)
d2 = final[['CUUR0000SACL1E', 'core_services', 'CUUR0000SAH1', 'food_energy']].loc['2011-01-01':]
# Rename columns
col_names = ['core_goods', 'core_services', 'shelter', 'food_energy']
d2.columns = col_names
d2['total'] = final['CUUR0000SA0'].loc['2011-01-01':]

d2.to_csv(data_dir / 'cpi_decomp.csv', index_label='date')

ltdate = dtxt(d2.index[-1])['mon1']
prdate = dtxt(d2.index[-13])['mon1']
cg = cont_subt(d2.core_goods.iloc[-1])
cs = cont_subt(d2.core_services.iloc[-1], style='end')
sh = cont_subt(d2.shelter.iloc[-1], style='end')
fe = cont_subt(d2.food_energy.iloc[-1], style='end')
tot = d2.total.iloc[-1]
cgpr = cont_subt(d2.core_goods.iloc[-13], style='end')
cspr = cont_subt(d2.core_services.iloc[-13], style='end')
shpr = cont_subt(d2.shelter.iloc[-13], style='end')
fepr = cont_subt(d2.food_energy.iloc[-13], style='end')
totpr = d2.total.iloc[-13]
text = (f'As of {ltdate}, core goods {cg} the overall non-seasonally-'+
        f'adjusted CPI inflation rate of {tot:.2f} percent, '+
        f'while core services excluding shelter {cs}. Shelter {sh}, '+
        f'and food and energy {fe}. One year prior, in {prdate}, '+
        f'the corresponding CPI inflation rate was {totpr:.2f} percent; '
        f'core goods {cgpr}, core services excluding shelter {cspr}, '+
        f'shelter {shpr}, and food and energy {fepr}.')
write_txt(text_dir / 'cpi_decomp.txt', text)
print(text)

### CPI chart

In [ ]:
# Series stored as a dictionary
series = {
    'CUSR0000SA0': 'ALL', 
    'CUSR0000SA0L1E': 'CORE'}

# Start year and end year
years = (1988, 2020)
df = bls_api(series, years, bls_key)

(df.pct_change(12).dropna() * 100).to_csv(data_dir / 'cpi.csv', index_label='date')

### CPI text

In [ ]:
s = pd.read_csv(data_dir / 'cpi.csv')
date = pd.to_datetime(s['date'].iloc[-1]).strftime('%B %Y')
core = s['CORE'].iloc[-1]
allitems = s['ALL'].iloc[-1]

text = ('\href{https://www.bls.gov/cpi/}{Consumer prices} increased by '+
        f'{allitems:.1f} percent over the year ending {date}, according '+
        'to the CPI for all urban consumers. Core inflation, which does '+
        'not include the more volatile food and energy prices, was '+
        f'{core:.1f} percent.')

write_txt(text_dir / 'cpi_main.txt', text)

text

### PPI

In [ ]:
ppi = (fred_df('PPIACO', start='1988')['VALUE'].pct_change(12) * 100).dropna()
ppi.to_csv(data_dir / 'ppi.csv', index_label='date')
node = end_node(ppi, 'green!80!blue', date=True)
write_txt(text_dir / 'ppi_node.txt', node)

ch = inc_dec_percent(ppi.iloc[-1])
prval = ppi.iloc[-13]
compare = compare_text(ppi.iloc[-1], prval, [1.0, 3.0, 5.0])
date = dtxt(ppi.index[-1])['mon1']
date2 = dtxt(ppi.index[-13])['mon1']

text = ('The Bureau of Labor Statistics \\href{https://www.bls.gov/ppi/}{reports} '+
        'prices producers receive for the various goods and services they produce. '+
        'The producer price index for all commodities (see {\color{green!80!blue}\\textbf{---}}) '+
        f'{ch} over the year ending {date}, {compare} '+
        f'the 12-month growth rate of {prval:.1f} percent '+
        f'in {date2}.')

write_txt(text_dir / 'ppi_main.txt', text)
print(text)

### Import/Export Price Index

In [ ]:
# Series stored as a dictionary
series = {'EIUIR': 'Imports', 
          'EIUIQ': 'Exports',
          'EIUIREXFUELS': 'ImpExFuels',
          'EIUIR10': 'ImpFuels',
          'EIUIQEXAG': 'ExpExAg',
          'EIUIQAG': 'ExpAg'}

# Start year and end year
years = (1988, 2020)
df = bls_api(series, years, bls_key)

df.to_csv(data_dir / 'mxpi_main.csv', index_label='date')

srs = ['Imports', 'Exports']
(df[srs].pct_change(12).dropna() * 100).to_csv(data_dir / 'mxpi.csv', index_label='date')

In [ ]:
df = pd.read_csv(data_dir / 'mxpi_main.csv', index_col='date')

df.index = pd.to_datetime(df.index)

data = (df.pct_change(12).dropna() * 100)

ltval = data['Imports'].iloc[-1]
ch = inc_dec_percent(ltval)
ltdate = dtxt(data.index[-1])['mon1']
chpr = inc_dec_percent(data['Imports'].iloc[-2])
prdate = dtxt(data.index[-2])['mon1']

s = series_info(data['Imports'])
m3y = s['three_year_mean']
lm = f", {s['last_matched']}" if s['days_since_match'] > 1000 else ''
compare = compare_text(ltval, m3y, [0.4, 2.0, 6.0])

text = ('According to the Bureau of Labor Statistics, US import '+
        'prices (see {\color{cyan!85!yellow}\\textbf{---}}) '+
        f'{ch} over the 12 months ending {ltdate}{lm}. '+
        f'Import prices {chpr} over the 12 months ending the month prior ({prdate}). '
        f'The latest value is {compare} the past-three-year average growth rate '+
        f'of {m3y:.1f} percent.')
write_txt(text_dir / 'mxpi_imp.txt', text)
print(text)

In [ ]:
ltval = data['Exports'].iloc[-1]
ch = inc_dec_percent(ltval)
ltdate = dtxt(data.index[-1])['mon1']
chpr = inc_dec_percent(data['Exports'].iloc[-2])
prdate = dtxt(data.index[-2])['mon1']

s = series_info(data['Exports'])
m3y = s['three_year_mean']
lm = f", {s['last_matched']}, and " if s['days_since_match'] > 1000 else ', '
compare2 = compare_text(ltval, m3y, [0.4, 2.0, 6.0])
also = 'also ' if compare2 == compare else ''

text = ('Prices of US exports (see {\color{red!25!orange}\\textbf{---}}) '+
        f'{ch} over the year ending {ltdate}{lm}{also}{compare2} '+
        f'its three-year moving average growth rate of {m3y:.1f} percent. '+
        f'Export prices {chpr} over the 12 months ending {prdate}.')
write_txt(text_dir / 'mxpi_exp.txt', text)
print(text)

### PCE Price Index

In [ ]:
data = pd.DataFrame()

df = fred_df('PCEPI', start='1988').pct_change(12)
data['PCE'] = df['VALUE'].dropna() * 100.0
df = fred_df('PCEPILFE', start='1988').pct_change(12)
data['CORE'] = df['VALUE'].dropna() * 100.0

data.to_csv(data_dir / 'pce_pi.csv', index_label='date')

ltdate = dtxt(data.index[-1])['mon1']
prdate = dtxt(data.index[-2])['mon1']
pryrdate = dtxt(data.index[-13])['mon1']
ltval = data.PCE.iloc[-1]
prval = data.PCE.iloc[-2]
pryrval = data.PCE.iloc[-13]
ltcore = data.CORE.iloc[-1]
prcore = data.CORE.iloc[-2]
pryrcore = data.CORE.iloc[-13]

text = (f'As of {ltdate}, PCE inflation, measured as the one year percent '+
        f'change in the overall index, is {ltval:.1f} percent, compared to '+
        f'{prval:.1f} percent in {prdate}, and {pryrval:.1f} percent in {pryrdate}. '+
        f'Core PCE inflation, excluding food and energy, was {ltcore:.1f} percent '+
        f'in {ltdate}, {prcore:.1f} percent in {prdate}, and {pryrcore:.1f} '+
        f'percent in {pryrdate}.')

write_txt(text_dir / 'pce_inf_basic.txt', text)

print(text)